# LLM Compiler Agent Cookbook

<a href="https://colab.research.google.com/github/run-llama/llama-hub/blob/main/llama_hub/llama_packs/agents/llm_compiler/llm_compiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**NOTE**: Full credits to the [source repo for LLMCompiler](https://github.com/SqueezeAILab/LLMCompiler). A lot of our implementation was lifted from this repo (and adapted with LlamaIndex modules).

In this cookbook, we show how to use our LLMCompiler agent implementation for various settings. This includes using some simple function tools to do math, but also to answer multi-part queries for more advanced RAG use cases over multiple documents.

We see that the LLMCompilerAgent is capable of parallel function calling, giving results much more quickly than sequential execution through ReAct.


In [ ]:
%pip install llama-index-agent-llm-compiler
%pip install llama-index-readers-wikipedia
%pip install llama-index-llms-openai

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.agent.llm_compiler import LLMCompilerAgentWorker

## Test LLMCompiler Agent with Simple Functions

Here we test the LLMCompilerAgent with simple math functions (add, multiply) to illustrate how it works.

In [ ]:
import json
from typing import Sequence, List

from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

### Define Functions

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

tools = [multiply_tool, add_tool]

In [ ]:
multiply_tool.metadata.fn_schema_str

'{"type": "object", "properties": {"a": {"title": "A", "type": "integer"}, "b": {"title": "B", "type": "integer"}}, "required": ["a", "b"]}'

### Setup LLMCompiler Agent

We import the `LLMCompilerAgentWorker` and combine it with the `AgentRunner`.

In [ ]:
from llama_index.core.agent import AgentRunner

In [ ]:
llm = OpenAI(model="gpt-4")

In [ ]:
callback_manager = llm.callback_manager

In [ ]:
agent_worker = LLMCompilerAgentWorker.from_tools(
    tools, llm=llm, verbose=True, callback_manager=callback_manager
)
agent = AgentRunner(agent_worker, callback_manager=callback_manager)

### Test out some Queries

In [ ]:
response = agent.chat("What is (121 * 3) + 42?")

> Running step 1ecbfd84-5440-4390-b954-a6c83b2dcb9a for task 8ac1971d-e9ee-4bf3-9e42-b6b07ab7c2da.
> Step count: 0
> Plan: 1. multiply(121, 3)
2. add($1, 42)
Thought: I can answer the question now.
3. join()<END_OF_PLAN>
Ran task: multiply. Observation: 363
Ran task: add. Observation: 405
Ran task: join. Observation: None
> Thought: The result of the operation is 405.
> Answer: 405


In [ ]:
response

AgentChatResponse(response='405', sources=[], source_nodes=[], is_dummy_stream=False)

In [ ]:
agent.memory.get_all()

[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is (121 * 3) + 42?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='405', additional_kwargs={})]

## Try out LLMCompiler for RAG

Now let's try out the LLMCompiler for RAG use cases. Specifically, we load a dataset of Wikipedia articles about various cities and ask questions over them.

### Setup Data

We use our `WikipediaReader` to load data for various cities.

In [ ]:
%pip install llama-index-readers-wikipedia wikipedia

In [ ]:
from llama_index.readers.wikipedia import WikipediaReader

In [ ]:
wiki_titles = ["Toronto", "Seattle", "Chicago", "Boston", "Miami"]

In [ ]:
city_docs = {}
reader = WikipediaReader()
for wiki_title in wiki_titles:
    docs = reader.load_data(pages=[wiki_title])
    city_docs[wiki_title] = docs

### Setup LLM + Service Context

In [ ]:
from llama_index.core import ServiceContext
from llama_index.llms.openai import OpenAI
from llama_index.core.callbacks import CallbackManager

llm = OpenAI(temperature=0, model="gpt-4")
service_context = ServiceContext.from_defaults(llm=llm)
callback_manager = CallbackManager([])

/var/folders/0g/wd11bmkd791fz7hvgy1kqyp00000gn/T/ipykernel_52010/432572999.py:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm)


### Define Toolset

In [ ]:
from llama_index.core import load_index_from_storage, StorageContext
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import VectorStoreIndex
import os

node_parser = SentenceSplitter()

# Build agents dictionary
query_engine_tools = []

for idx, wiki_title in enumerate(wiki_titles):
    nodes = node_parser.get_nodes_from_documents(city_docs[wiki_title])

    if not os.path.exists(f"./data/{wiki_title}"):
        # build vector index
        vector_index = VectorStoreIndex(
            nodes,
            service_context=service_context,
            callback_manager=callback_manager,
        )
        vector_index.storage_context.persist(
            persist_dir=f"./data/{wiki_title}"
        )
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=f"./data/{wiki_title}"),
            service_context=service_context,
            callback_manager=callback_manager,
        )
    # define query engines
    vector_query_engine = vector_index.as_query_engine()

    # define tools
    query_engine_tools.append(
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name=f"vector_tool_{wiki_title}",
                description=(
                    "Useful for questions related to specific aspects of"
                    f" {wiki_title} (e.g. the history, arts and culture,"
                    " sports, demographics, or more)."
                ),
            ),
        )
    )

### Setup LLMCompilerAgent

In [ ]:
from llama_index.core.agent import AgentRunner
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4")
agent_worker = LLMCompilerAgentWorker.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    callback_manager=callback_manager,
)
agent = AgentRunner(agent_worker, callback_manager=callback_manager)

### Test out Queries

In [ ]:
response = agent.chat(
    "Tell me about the demographics of Miami, and compare that with the demographics of Chicago?"
)
print(str(response))

> Running step 7010cd56-3e69-4e25-8875-1ae0ab644e7d for task 1ec47644-e7e5-459e-81b4-2cb2f8105912.
> Step count: 0
> Plan: 1. vector_tool_Miami("demographics")
2. vector_tool_Chicago("demographics")
3. join()<END_OF_PLAN>
Ran task: vector_tool_Chicago. Observation: Chicago's demographics are diverse and have changed significantly over time. The city's industrial working class was initially formed by various ethnic groups, with a significant influx of African Americans from the American South in the early 20th century. The city also has a notable Bosnian population that arrived mainly in the 1990s and 2000s. Most of the foreign-born population in Chicago were born in Mexico, Poland, and India. 

As of July 2019, the largest racial or ethnic group in Chicago is non-Hispanic White, making up 32.8% of the population, followed by Blacks at 30.1% and the Hispanic population at 29.0%. The city also has the third-largest LGBT population in the United States, with an estimated 7.5% of the adult

In [ ]:
response = agent.chat(
    "Is the climate of Chicago or Seattle better during the wintertime?"
)
print(str(response))

> Running step c5ce4ccc-80a7-453b-8ebc-b3aeedfdb338 for task 16e3d4ca-7cab-4d0c-bbfa-225a09680ccd.
> Step count: 0
> Plan: 1. vector_tool_Chicago("climate during wintertime")
2. vector_tool_Seattle("climate during wintertime")
3. join()<END_OF_PLAN>
Ran task: vector_tool_Seattle. Observation: During wintertime, Seattle experiences cool, wet weather. Extreme cold temperatures, below about 15 °F or -9 °C, are rare due to the moderating influence of the adjacent Puget Sound, the greater Pacific Ocean, and Lake Washington. The city is often cloudy due to frequent storms and lows moving in from the Pacific Ocean, and it has many "rain days". However, the precipitation is often a light drizzle rather than heavy rainfall.
Ran task: vector_tool_Chicago. Observation: During wintertime, the city experiences relatively cold and snowy conditions. Blizzards can occur, as they did in winter 2011. The normal winter high from December through March is about 36 °F (2 °C). January and February are the c